In [1]:
import json

In [2]:
class Gates:
    
    def NOT(self, tup):
        x = tup[0]
        if x is 1:
            return 0
        return 1
       
    def OR(self, tup):
        x, y = tup
        if x == y and x == 0:
            return 0
        return 1
    
    def AND(self, tup):
        x, y = tup
        if x == y and x == 1:
            return 1
        return 0
    
    def XOR(self, tup):
        x, y = tup
        if x == y:
            return 1
        return 0
    
    def NAND(self, tup):
        x, y = tup
        return self.NOT(self.AND(x,y))
   
    def XNOR(self, tup):
        x, y = tup
        return self.NOT(self.XOR(x,y))


In [57]:

def load(filename="json/f.add.json"):
    with open(filename) as json_file:
        json_circuits = json.load(json_file)

    for json_circuit in json_circuits['circuits']:
        circuit = Circuit(json_circuit)
        circuit.printall()
        break

class Circuit:
    def __init__(self,circuitDict):
        self.name = None
        self.alice = None
        self.bob = None
        self.gates = None
        self.out = None
        # processes json.
        self.processCircuit(circuitDict)

    def processCircuit(self,circuit):
        for key in self.__dict__.keys():
            setattr(self,key,circuit[key])
        self.raw = circuit
            
    def compute(self,inputs):
        # initiate gates
        gates = Gates()
        # creates stores.
        limit = max(self.out)
        outputs = [0 for i in range(limit+1)]
        
        # store inputs
        for i in range(len(inputs)):
            outputs[i+1] = inputs[i]
        
        # iterate through the gate operations
        for gate in self.gates:
            # load logic gate
            logic = getattr(gates,gate['type'])
            # load inputs
            parameters = tuple([outputs[i] for i in gate['in']])
            # get output
            output = logic(parameters)
            # store result
            outputs[gate['id']] = output

        return [outputs[i] for i in self.out]
    
    def printall(self):
        print(self.name)
        for alice in self.perms(len(self.alice)):
            for bob in self.perms(len(self.bob)):
                inputs = alice + bob
                pr = "Alice"+str(self.alice)+" = " 
                for i in alice:
                    pr += str(i) + " "
                pr += "  Bob"+str(self.bob)+" = "
                for i in bob:
                    pr += str(i) + " "
                pr += "  Outputs"+str(self.out)+" = "
                for i in self.compute(inputs):
                    pr += str(i) + " "
                print(pr)
    
    @staticmethod
    def perms(n):
        if not n:
            return
        entries = []
        for i in range(2**n):
            s = bin(i)[2:]
            s = "0" * (n-len(s)) + s
            ent = [int(i) for i in s]
            entries.append(ent)
        return entries
        
load()

1-bit full adder Cin, A, B -> Sum, Cout
Alice[1, 2] = 0 0   Bob[3] = 0   Outputs[5, 11] = 0 0 
Alice[1, 2] = 0 0   Bob[3] = 1   Outputs[5, 11] = 1 0 
Alice[1, 2] = 0 1   Bob[3] = 0   Outputs[5, 11] = 1 0 
Alice[1, 2] = 0 1   Bob[3] = 1   Outputs[5, 11] = 0 1 
Alice[1, 2] = 1 0   Bob[3] = 0   Outputs[5, 11] = 1 1 
Alice[1, 2] = 1 0   Bob[3] = 1   Outputs[5, 11] = 0 0 
Alice[1, 2] = 1 1   Bob[3] = 0   Outputs[5, 11] = 0 0 
Alice[1, 2] = 1 1   Bob[3] = 1   Outputs[5, 11] = 1 1 
